In [2]:
# read the data and turn it into a dataframe
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
import pandas as pd
import os

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "1976_Sep2016_USPTOgrants_smiles.rsmi"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

dataFrame= pd.read_csv("1976_Sep2016_USPTOgrants_smiles.rsmi", delimiter='\t',low_memory=False)
#Delete columns that are not the reactions or the yield
columns_to_delete = ["PatentNumber", "ParagraphNum", "Year", "TextMinedYield"]
dataFrame.drop(columns=columns_to_delete, inplace=True)
#Separate the Reactants from the Products
new_columns = dataFrame['ReactionSmiles'].str.split('>', expand=True)
new_columns = new_columns.rename(columns={0: 'Reactant 1', 2 : "Product 1"})
dataFrame = pd.concat([new_columns, dataFrame.iloc[:, 1:]], axis=1)
dataFrame.drop(columns=1, inplace=True)
#Separate the reactants from each other
reactants_split = dataFrame['Reactant 1'].str.split("\.", expand = True)
reactants_split.columns = [f'Reactant {i+1}' for i in range(reactants_split.shape[1])]
dataFrame = pd.concat([reactants_split, dataFrame.iloc[:, 1:]], axis=1)
#Separate the Products from each other
products_split = dataFrame['Product 1'].str.split("\.", expand = True)
products_split.columns = [f'Product {i+1}' for i in range(products_split.shape[1])]
dataFrame = pd.concat([products_split, dataFrame.iloc[:, 1:]], axis=1)
dataFrame = dataFrame.loc[:, ~dataFrame.columns.duplicated(keep='first')]
dataFrame.head()


Current Directory: /Users/Travail/git/Amazing-Project
ReactionSmiles	PatentNumber	ParagraphNum	Year	TextMinedYield	CalculatedYield



,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,...,Reactant 53,Reactant 54,Reactant 55,Reactant 56,Reactant 57,Reactant 58,Reactant 59,Reactant 60,Reactant 61,CalculatedYield
0,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
1,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
2,[CH2:10]([S:14]([O:3][CH2:2][CH2:1][Cl:4])(=[O...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
3,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
4,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN


In [3]:
"""
This Cell groups the functions used to clean the dataframe to be able to use it later
"""
# Remove atome mapping numbers from smiles
import re

def remove_atom_mapping(smiles):
    """
    Remove atom mapping numbers from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers.
    Returns:
    - smiles_without_mapping (str): The SMILES notation without atom mapping numbers.
    """
    # Define a regular expression pattern to match atom mapping numbers
    pattern = r':\d+'
    smiles_without_mapping = re.sub(pattern, '', smiles)
    return smiles_without_mapping

dataFrame = dataFrame.astype(str)
columns_to_process = dataFrame.columns[:-1]
for column in columns_to_process:
    dataFrame[column] = dataFrame[column].apply(remove_atom_mapping)
    
#Remove the percentage symbol
def remove_percent_symbol(value):
    """
    Remove the '%' symbol from a percentage value.
    Args:
    - value (str): The percentage value with the '%' symbol.
    Returns:
    - value_without_percent (str): The percentage value without the '%' symbol.
    """
    return value.replace('%', '')

# Apply the remove_percent_symbol function to the "PercentageColumn" column
dataFrame['CalculatedYield'] = dataFrame['CalculatedYield'].apply(remove_percent_symbol)

In [4]:
# Prompt the user to enter the molecule that he wants to form
print ("you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base")
string_input_mol = input("Enter the molecule that you want to form: ")

you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base


Enter the molecule that you want to form:  bromide


In [5]:
# function to determine if the user enters the SMILES notation or the usual name of the molecule
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    if Chem.MolFromSmiles(smiles)== None:
        return False
    else:
        return True

# Check if the entered string is a valid SMILES notation
if is_smiles(string_input_mol):
    print(f"'{string_input_mol}' is a valid SMILES notation.")
else:
    print(f"'{string_input_mol}' is not a valid SMILES notation.")

'bromide' is not a valid SMILES notation.


[19:29:51] SMILES Parse Error: syntax error while parsing: bromide
[19:29:51] SMILES Parse Error: Failed parsing SMILES 'bromide' for input: 'bromide'


In [6]:
!pip install pubchempy

In [7]:
# function to convert name into SMILES

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information. Please try with the SMILE of the molecule")
            return None
    except:
        print("Error: Unable to retrieve molecule information. Please try again")
        return None

# Call the function to convert the molecule name to SMILES notation
if is_smiles(string_input_mol)== False:
    string_input_mol = name_to_smiles(string_input_mol)
    if string_input_mol:
        print(f"SMILES notation for {string_input_mol}: {string_input_mol}")
else:
    print (string_input_mol)

[19:29:54] SMILES Parse Error: syntax error while parsing: bromide
[19:29:54] SMILES Parse Error: Failed parsing SMILES 'bromide' for input: 'bromide'


SMILES notation for [Br-]: [Br-]


In [8]:
# Function that returns the row of the elements formation reaction
def compare_molecule_with_data(element):
    return ''.join(element.split()).lower() == ''.join(string_input_mol.split()).lower()

# Initialize a variable to store the row number
found_row_number = None
rows=[]
# Iterate over the last three columns of the DataFrame
for column_name in dataFrame.columns[0 : 128]:
    column = dataFrame[column_name]
    # Check if any element in the column matches the input molecule
    for index, value in column.items():
        if compare_molecule_with_data(value):
            # Store the row number where the molecule is found
            found_row_number = index
            rows.append(found_row_number)
    else:
        continue  # Continue to the next column if molecule not found in current column

# Check if the molecule was found and return the row number
if found_row_number is not None:
   print(rows)
else:
    print("The product is not in the database")


[612, 614, 846, 1409, 1531, 1683, 5626, 5986, 6179, 6269, 6376, 6379, 6380, 6383, 6558, 6584, 6878, 7112, 7620, 7920, 7921, 8049, 8214, 8511, 8648, 8649, 9536, 9538, 9551, 9973, 10174, 10175, 10270, 10271, 10275, 10276, 10966, 11245, 12028, 12841, 13005, 13011, 13015, 13428, 13430, 13434, 13705, 13706, 14595, 14901, 14902, 14910, 15666, 15667, 15671, 15672, 15673, 15674, 15801, 15834, 16126, 16347, 16348, 16349, 16350, 16351, 16352, 16354, 17246, 17247, 18187, 19078, 19079, 19080, 19704, 19705, 20298, 20478, 20843, 21368, 21422, 21423, 21424, 21425, 21426, 21856, 22607, 22608, 23074, 23075, 23076, 23082, 23083, 23084, 23085, 23147, 23889, 24464, 24556, 24824, 26180, 26382, 26705, 26706, 27505, 27506, 27658, 27659, 27699, 27701, 27702, 28158, 28548, 28549, 28717, 28718, 28822, 28918, 28924, 28928, 28930, 28931, 29127, 29128, 29129, 29134, 29561, 30183, 30184, 30415, 30417, 30787, 30788, 30899, 31271, 31523, 31524, 31694, 31715, 31889, 32510, 32511, 33007, 33733, 33735, 33736, 33750, 337

In [13]:
"""
This code prints the yield and shows the image of the reaction corresponding
"""
# Convert the 'Yield' column to float
dataFrame['CalculatedYield'] = dataFrame['CalculatedYield'].astype(float)
# Assuming 'rows_to_compare' is a list containing the row numbers to compare
# Subset the DataFrame to include only the specified rows
subset_df = dataFrame.iloc[rows]
print(subset_df)
# Find the index of the row with the highest yield
max_yield_row_index = subset_df['CalculatedYield'].idxmax()
# Retrieve the row with the highest yield
max_yield_row = dataFrame.loc[max_yield_row_index]
print("Row with the highest yield:")
print(max_yield_row)

# print the reaction
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
#Create a Dataframe where the reaction is in one column
dataFrameImage= pd.read_csv("1976_Sep2016_USPTOgrants_smiles.rsmi", delimiter='\t',low_memory=False)
columns_to_delete = ["PatentNumber", "ParagraphNum", "Year", "TextMinedYield"]
dataFrameImage.drop(columns=columns_to_delete, inplace=True)
dataFrameImage["ReactionSmiles"] = dataFrameImage["ReactionSmiles"].apply(remove_atom_mapping)
# Create a reaction SMILES string
reaction_smiles = dataFrameImage.iloc[max_yield_row_index, 0]
# Parse the reaction SMILES string
reaction = Chem.AllChem.ReactionFromSmarts(reaction_smiles)
# Draw the reaction
reaction_image = Draw.ReactionToImage(reaction)

# Save or display the image
reaction_image.show()

        Product 1                                          Product 2  \
612         [Br-]  [C]([CH2][CH2][P+]([C]1[CH]=[CH][CH]=[CH][CH]=...   
614         [Br-]  [C]1([CH2][CH2][CH2][CH2][P+]([C]2[CH]=[CH][CH...   
846         [Br-]  [Br][CH2][CH2][N+]1[C]2[C](=[CH][CH]=[CH][CH]=...   
1409        [Br-]                                             [PH4+]   
1531        [Br-]  [NH2][C]1[CH]=[CH][CH]=[CH][N+]=1[CH2][C]1[CH]...   
...           ...                                                ...   
750530      [Br-]                                              [Br-]   
1564381     [Br-]  [C]1([CH2][N+]2([CH2][CH]=[CH2])[CH]=[CH][C]([...   
90025       [Br-]  [O]=[C]([C]1[CH]=[CH][CH]=[CH][CH]=1)[CH2][CH]...   
215239      [Br-]                                              [Br-]   
216008      [Br-]                                              [Br-]   

                                                 Product 3  \
612                                                     4|   
614        